In [1]:
%pwd

'/home/j-i13a103'

In [1]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

2025-08-05 05:16:59.552673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-05 05:16:59.570504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754371019.592156  477275 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754371019.598617  477275 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754371019.615752  477275 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
# 2) 디바이스 설정: CUDA가 사용 가능하면 GPU('cuda')를, 그렇지 않으면 CPU('cpu')를 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")  # 선택된 디바이스(예: cuda 또는 cpu)를 콘솔에 출력하여 확인

Using device: cuda


In [9]:
BASE_PATH = "/home/j-i13a103/tts/run/training"

# xtts_config 경로 설정
CONFIG_PATH = BASE_PATH + "/GPT_XTTS_v2.0_SSOKDAK_FT-August-04-2025_08+49AM-0000000/config.json"

# 학습된 모델의 vocab.json 설정
TOKENIZER_PATH = BASE_PATH + "/XTTS_v2.0_original_model_files/vocab.json"

# best model 가져오기
XTTS_CHECKPOINT = BASE_PATH + "/GPT_XTTS_v2.0_SSOKDAK_FT-August-04-2025_08+49AM-0000000/best_model.pth"

# 레퍼런스 넣기
SPEAKER_REFERENCE = "/home/j-i13a103/tts/korean-single-speaker-datasets/wavs/1_0000.wav"

# 결과 음성 파일
OUTPUT_WAV_PATH = "/home/j-i13a103/tts/finetuning-result/output_v1_temp07.wav"

In [17]:
print("Loading model...")

config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model.to(device)

Loading model...


Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(

In [23]:
# speaker-encoder 제거하기
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])

# 파일 저장하기
# torch.save({"gpt_latent": gpt_cond_latent.cpu(), "speaker_embedding": speaker_embedding.cpu()}, "/home/j-i13a103/tts/finetuning-result/model/fixed_speaker.pt")

# 양자화

In [7]:
import torch.nn as nn
from torch.quantization import quantize_dynamic

In [20]:
# 모델 카피
new_model = Xtts.init_from_config(config)
new_model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
new_model.to(device)

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttentionLegacy()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(

In [21]:
# GPT2Model 양자화
new_model.gpt.gpt = quantize_dynamic(
    model.gpt.gpt,
    {nn.Linear},
    dtype=torch.qint8
)

# HiFi-GAN Decoder 양자화
new_model.hifigan_decoder = quantize_dynamic(
    model.hifigan_decoder,
    {nn.Conv1d, nn.Linear},
    dtype=torch.qint8
)

In [22]:
# 특정 부분만 양자화한 모델로 추론하기
new_model.to(device)
new_model.eval()

In [24]:
out = new_model.inference(
    text="안녕, 오늘 기분 어때?",
    language="ko",
    gpt_cond_latent=gpt_cond_latent,
    speaker_embedding=speaker_embedding,
    temperature=0.7
)

torchaudio.save("/home/j-i13a103/tts/finetuning-result/output_quantized_v1.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)

In [26]:
# 양자화된 모델 저장하기
NEW_MODEL = "/home/j-i13a103/tts/finetuning-result/model/xtts_quantized_model.pth"

torch.save(new_model.state_dict(), NEW_MODEL)

# 크기 비교하기
print("원본 체크포인트:", os.path.getsize(XTTS_CHECKPOINT)/1e9, "GB")

print("양자화 모델 :", os.path.getsize(NEW_MODEL)/1e9, "GB")

원본 체크포인트: 5.607969782 GB
양자화 모델 : 1.861591649 GB


양자화 state-dict : 3.377433587 GB


In [14]:
# 전체 모델을 양자화하기

quantized_model = quantize_dynamic(
    model,
    {nn.Linear, nn.Conv1d},
    dtype=torch.qint8
)

quantized_model.to("cpu")

quantized_model.eval()

print("Quantized model loaded and ready for inference.")

Quantized model loaded and ready for inference.


In [15]:
# 양자화된 모델 성능 확인하기
print("Inference...")

out = quantized_model.inference(
    "안녕, 오늘 하루 어땠어?",
    "ko",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7
)

# 오디오 출력
torchaudio.save("/home/j-i13a103/tts/finetuning-result/output_quantized_v1.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)

print("Saved output.")

Inference...


RuntimeError: apply_dynamic is not implemented for this packed parameter type